<a href="https://colab.research.google.com/github/vainamonen/dw_matrix_road_sign/blob/master/day_5_road_sign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/Colab Notebooks/matrix/matrix_three

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three


In [4]:
ls data/

logs/  signnames.csv  test.p  train.p


In [5]:
!pip install hyperopt

In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
    X_train, 
    y_train,
    batch_size=params_fit.get('batch_size', 128),
    epochs=params_fit.get('epochs', 5),
    verbose=params_fit.get('verbose', 1),
    validation_data=params_fit.get('validation_data', (X_train, y_train)),
    callbacks= [tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
    
    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax'),
])

In [17]:
train_and_predict(get_cnn_v5(input_shape, num_classes))

Epoch 1/5
272/272 [==============================] - 5s 20ms/step - loss: 2.5118 - accuracy: 0.2941 - val_loss: 1.0381 - val_accuracy: 0.6695
Epoch 2/5
272/272 [==============================] - 5s 17ms/step - loss: 0.8879 - accuracy: 0.7149 - val_loss: 0.2625 - val_accuracy: 0.9205
Epoch 3/5
272/272 [==============================] - 5s 17ms/step - loss: 0.4276 - accuracy: 0.8651 - val_loss: 0.0875 - val_accuracy: 0.9749
Epoch 4/5
272/272 [==============================] - 5s 17ms/step - loss: 0.2719 - accuracy: 0.9162 - val_loss: 0.0693 - val_accuracy: 0.9788
Epoch 5/5
272/272 [==============================] - 5s 17ms/step - loss: 0.1990 - accuracy: 0.9393 - val_loss: 0.0400 - val_accuracy: 0.9883


0.97437641723356

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax'),
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
  )

  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [15]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4791640385788194, 'dropout_cnn_block_three': 0.44694598812806796, 'dropout_cnn_block_two': 0.4318330270230376, 'dropout_dense_block_one': 0.5543529671064471, 'dropout_dense_block_two': 0.40340761292972427}
accuracy=0.8945578336715698
{'batch_size': 140.0, 'dropout_cnn_block_one': 0.443593951586454, 'dropout_cnn_block_three': 0.4543603268482915, 'dropout_cnn_block_two': 0.465800966034907, 'dropout_dense_block_one': 0.45141950291421223, 'dropout_dense_block_two': 0.631949145275972}
accuracy=0.7863945364952087
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.40305359498522, 'dropout_cnn_block_three': 0.40518038552671204, 'dropout_cnn_block_two': 0.45185136932477327, 'dropout_dense_block_one': 0.6707687299852798, 'dropout_dense_block_two': 0.6628702654648684}
accuracy=0.6217687129974365
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.3769410227221897, 'dropout_cnn_block_three': 0.34568352104012096, 'dropout_cnn_block_two': 0.445563330299194